In [113]:
# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
import pandas as pd
import os.path
from os import path
from matplotlib import pyplot
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
import functools 


In [2]:
import import_ipynb
from common import splitDataset
from common import splitDatasetLabels
from common import normDataset
from common import evaluatePerf

importing Jupyter notebook from common.ipynb


In [108]:
def buildCNN(strides, kernels, layers):
    n_timesteps, n_features, n_outputs = 1024, 1, 1
    model = Sequential()
    layer = 0
    for s,k,l in zip(strides,kernels,layers):
        if (layer == 0): model.add(Conv1D(filters=l, kernel_size=k, strides=s, activation='relu', input_shape=(n_timesteps,1)))
        else: model.add(Conv1D(filters=l, kernel_size=k, strides=s, activation='relu'))
        
#     model.add(Dropout(0.5))
#     model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(n_outputs, activation='relu'))
    model.compile(loss='mse', 
                  optimizer='adam', 
#                   optimizer=keras.optimizers.Adagrad(learning_rate=100.0),
                  metrics=['accuracy'])
    return model
    

In [99]:
def trainCNN(model, trainX, trainY, epochs):
    # fit network
    model.fit(trainX, trainY, epochs=epochs, steps_per_epoch=trainY.shape[0], verbose=1)
    
    return model

In [123]:
# Handle raw data

datadir = '/scratch/mattfel/raw_timetool/u1/coffee/2dtimetool_simulation_data/raw/'

raw_dataset = pd.DataFrame()
for fileId in range(1000,2000):
    fileType = 'interference'
    if (fileId % 100 == 0): print('at %d' % fileId)
    f = datadir + '/chirp-2000_interferedelay1650_photonen6.0_carriertagdiamond_nfibers109_netalon1_1.00_1.00_%s.out.%d' % (fileType,fileId)
    if (path.exists(f)):
        rawdata = pd.read_csv(f, 
                          skiprows = 7, usecols = [i for i in range(1024)], header=None,
                          na_values = "?", comment='\t',
                          sep="\t|,", skipinitialspace=True)
        raw_dataset = raw_dataset.append(rawdata)
    else:
        print('file %d does not exist!' % fileId)
    
# raw_dataset = tf.stack([raw_dataset], axis=2)


raw_labels = pd.DataFrame()
for fileId in range(1000,2000):
    fileType = 'fibermap'
    if (fileId % 100 == 0): print('at %d' % fileId)
    f = datadir + '/chirp-2000_interferedelay1650_photonen6.0_carriertagdiamond_nfibers109_netalon1_1.00_1.00_%s.out.%d' % (fileType,fileId)
    if (path.exists(f)):
        rawdata = pd.read_csv(f, 
                          skiprows = 1, usecols = [6], 
                          na_values = "?", comment='\t',
                          sep="\t|,", skipinitialspace=True)
        raw_labels = raw_labels.append(rawdata)
    else:
        print('file %d does not exist!' % fileId)


# print(raw_labels)
# print(raw_dataset.iloc[109])

at 1000


/home/mattfel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


at 1100
at 1200
at 1300
at 1400
at 1500
at 1600
at 1700
at 1800
at 1900


In [126]:
print(raw_labels.shape)
print(raw_dataset.shape)

(109000, 1)
(109000, 1024)


In [127]:
train_dataset, test_dataset, train_labels, test_labels = splitDatasetLabels(raw_dataset, raw_labels, 0.2)
train_dataset = tf.stack([train_dataset], axis=2)
test_dataset = tf.stack([test_dataset], axis=2)
# train_labels = tf.stack([train_labels], axis=2)
# test_labels = tf.stack([test_labels], axis=2)

In [ ]:
show_perf = True
load_prior_model = False
train_model = True
strong_data_sfx = "_strong"

strides=[1]
kernels=[16]
chans=[8]
model = buildCNN(strides, kernels, chans)
model.summary()
model = trainCNN(model, train_dataset, train_labels, 1000)
if (show_perf):
    preds = model.predict(test_dataset, steps=1)
    if (show_perf): evaluatePerf(test_labels, preds)

        
strides=[1]
kernels=[64]
chans=[8]
model = buildCNN(strides, kernels, chans)
model.summary()
model = trainCNN(model, train_dataset, train_labels, 1000)
if (show_perf):
    preds = model.predict(test_dataset, steps=1)
    if (show_perf): evaluatePerf(test_labels, preds)
        
        
strides=[2,2,2]
kernels=[16,16,16]
chans=[4,4,4]
model = buildCNN(strides, kernels, chans)
model.summary()
model = trainCNN(model, train_dataset, train_labels, 1000)
if (show_perf):
    preds = model.predict(test_dataset, steps=1)
    if (show_perf): evaluatePerf(test_labels, preds)



Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_42 (Conv1D)           (None, 1009, 8)           136       
_________________________________________________________________
flatten_19 (Flatten)         (None, 8072)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 8073      
Total params: 8,209
Trainable params: 8,209
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
 1446/87200 [..............................] - ETA: 140:00:14 - loss: 2004840.6874 - accuracy: 0.0000e+00

In [95]:
preds = model.predict(test_dataset, steps=1)
if (show_perf): evaluatePerf(test_labels, preds)

MSE, MAE, r2: 1634321.932250 955.822019 -0.229121
